In [ ]:
# Import necessary libraries
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# Load environment variables from .env file
load_dotenv()

# Initialize Langchain models with appropriate parameters
gemini_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)
nvidia_model = ChatNVIDIA(model="meta/llama-3.2-3b-instruct", temperature=0.1)

# Example usage of the models
gemini_response = gemini_model.invoke("What is the capital of France?")
print(f"Gemini Output: {gemini_response.content}")

nvidia_response = nvidia_model.invoke("What is the capital of France?")
print(f"NVIDIA Output: {nvidia_response.content}")

In [ ]:
# Trying different prompt templates in langchain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate

# Prompt Templates
prompt_template = PromptTemplate(
    template="What is the capital of {country}?",
    input_variables=["country"]
)

# Chat Prompt Template
chat_prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "You are a helpful assistant for python coding."),
        ("user", "{question}")
    ],
    input_variables=["question"]
)

# Few Shot Prompt Template
examples = [
    {"input": "Translate 'hello' to Spanish.", "output": "Hola."},
    {"input": "Translate 'thank you' to French.", "output": "Merci."}
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Translate the following phrases:\n",
    suffix="Input: {query}\nOutput:",
    input_variables=["query"]
)

# Example usage of the prompt templates
print(f"Prompt Template: {prompt_template.invoke({'country': 'France'}).text}")
print(f"Chat Prompt Template: {chat_prompt_template.invoke({'question': 'What is the capital of France?'})}")
print(f"Few Shot Prompt Template: {few_shot_prompt_template.invoke({'query': 'Translate Hello world into Hindi'}).text}")

In [ ]:
# Prompt Templates + Models
gemini_prompt_template = prompt_template | gemini_model
nvidia_prompt_template = prompt_template | nvidia_model

# Chat Prompt Templates + Models
gemini_chat_prompt_template = chat_prompt_template | gemini_model
nvidia_chat_prompt_template = chat_prompt_template | nvidia_model

# Few Shot Prompt Templates + Models
gemini_few_shot_prompt_template = few_shot_prompt_template | gemini_model
nvidia_few_shot_prompt_template = few_shot_prompt_template | nvidia_model

# Example usage of the combined prompt templates and models
gemini_combined_response = gemini_prompt_template.invoke({"country": "India"})
nvidia_combined_response = nvidia_prompt_template.invoke({"country": "India"})
print(f"Gemini Combined Output: {gemini_combined_response.content}")
print(f"NVIDIA Combined Output: {nvidia_combined_response.content}")

# Example usage of the chat prompt templates with models
gemini_chat_response = gemini_chat_prompt_template.invoke({"question": "Write python code to reverse a string."})
nvidia_chat_response = nvidia_chat_prompt_template.invoke({"question": "Write python code to reverse a string?"})
print(f"Gemini Chat Output: {gemini_chat_response.content}")
print(f"NVIDIA Chat Output: {nvidia_chat_response.content}")

# Example usage of the few shot prompt templates with models
gemini_few_shot_response = gemini_few_shot_prompt_template.invoke({"query": "Translate Hello world into Hindi"})
nvidia_few_shot_response = nvidia_few_shot_prompt_template.invoke({"query": "Translate Hello world into Hindi"})
print(f"Gemini Few Shot Output: {gemini_few_shot_response.content}")
print(f"NVIDIA Few Shot Output: {nvidia_few_shot_response.content}")

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# Message PlaceHolders
# Define a chat prompt template with messages
message_placeholder_prompt = ChatPromptTemplate(
    messages= [
        ("system", "You are a friendly Chatbot."),
        ("placeholder", "{chat_history}"),
        ("user", "{user_query}"),  
    ],
    input_variables=["country", "chat_history"]
)

chat_history = [
    HumanMessage(content="Hii My name is Abhijit."),
    AIMessage(content="Hello there. Nice to meet you, Abhijit!"),
    HumanMessage(content="I am from India and I am learning about Langchain."),
    AIMessage(content="Ohh that's great. Langhain is a powerful framework for building applications with language models. What would you like to know about it?"),
]

final_response = message_placeholder_prompt.invoke({
    "user_query": "What is my name?",
    "chat_history": chat_history
})

final_response

In [ ]:
gemini_with_placeholder_memory = message_placeholder_prompt | gemini_model
nvidia_with_placeholder_memory = message_placeholder_prompt | nvidia_model

# Example usage of the models with message placeholders
print(f"Gemini with Placeholder Memory Output: {gemini_with_placeholder_memory.invoke({'chat_history': chat_history, 'user_query': 'What is my name?'}).content}")
print(f"NVIDIA with Placeholder Memory Output: {nvidia_with_placeholder_memory.invoke({'chat_history': chat_history, 'user_query': 'What is my name?'}).content}")

In [ ]:
from typing import TypedDict, Annotated, Optional, Literal
from pydantic import BaseModel, Field

# With Structured Output
# Define a structured output model using Pydantic
class UserProfile(BaseModel):
    name: Annotated[str, Field(description="The user's name")]
    age: Optional[Annotated[int, Field(description="The user's age, if provided", ge=0, le=120)]] = None
    email: Optional[Annotated[str, Field(description="The user's email address")]] = None
    gender: Annotated[Literal["Male", "Female"], Field(description="The user's gender. If not provided then use your best guess")]

# Create a prompt template for structured output  
gemini_structured = gemini_model.with_structured_output(UserProfile)
prompt_template_structured = PromptTemplate(
    template="fetch the user details like name, age and email from the user query: {query}",
    input_variables=["query"]
)

# Combine the prompt template with the model for structured output
structured_chain = prompt_template_structured | gemini_structured
structured_response = structured_chain.invoke({"query": "My name is Devika, I am 25 years old."})
structured_response = structured_response.model_dump()  # Convert to dictionary for easier access

# Print the structured response
print(f"Structured Response: {structured_response}")

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser, JsonOutputParser

# Define a Pydantic model for product details
class Product(BaseModel):
    name: Annotated[str, Field(description="The name of the product")]
    price: Annotated[float, Field(description="The price of the product in USD")]
    in_stock: Annotated[bool, Field(description="Whether the product is in stock")]
    
# Define output parsers
pydantic_parser = PydanticOutputParser(pydantic_object=Product)

# Define a prompt template for extracting product details
prompt_template_product = PromptTemplate(
    template="""Extract product details from the following text: 
    {text}
    format instructions: {format_instructions}""",
    input_variables=["text"],
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
)

# Create a chain that combines the prompt template and the model with the Pydantic parser
product_chain = prompt_template_product | gemini_model | pydantic_parser
product_response = product_chain.invoke({
    "text": "The product is a smartphone named 'Galaxy S21', priced at 799.99 USD, and it is currently out of stock."
})

print(f"Product Details: {product_response.model_dump()}")

In [ ]:
# Simple String Output Parser
simple_string_parser = StrOutputParser()
simple_string_template = PromptTemplate(
    template="What is the capital of {country}?",
    input_variables=["country"]
)

# Combine the simple string template with the model
simple_string_chain = simple_string_template | gemini_model | simple_string_parser
simple_string_response = simple_string_chain.invoke({"country": "France"})

print(f"Simple String Output: {simple_string_response}")

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chains

# Simple Chain
simple_prompt = PromptTemplate(
    template = "Describe about the topic in brief: {topic}",
    input_variables=["topic"]
)

parser = StrOutputParser()
simple_chain = simple_prompt | gemini_model | parser
simple_chain_response = simple_chain.invoke({"topic": "Blackhole"})
print(f"Simple Chain Response: {simple_chain_response}")



In [ ]:
# Sequential Chain
summary_prompt = PromptTemplate(
    template="Summarize the following text: {text}",
    input_variables=["text"]
)

translate_prompt = PromptTemplate(
    template="Translate the following text to Hindi: {text}",
    input_variables=["text"]
)

sequential_chain = (
    summary_prompt | gemini_model | StrOutputParser()
) | (
    translate_prompt | gemini_model | StrOutputParser()
)

sequential_chain_response = sequential_chain.invoke({
    "text": "Hello My name is Abhijit Maharana and I am 22 years old. I am learning about Langchain and Langgrpah to build RAG applications and Agentic AI models."
})

print(sequential_chain_response)

In [ ]:
from langchain.schema.runnable import RunnableParallel
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Parallel Chain
question_prompt = PromptTemplate(
    template="""Make 3 questions and answers from the given text: {text}
    Format: 
    Question: 
    Answer:""",
    input_variables=["text"]
)

summary_prompt = PromptTemplate(
    template="Summarize the following text: {text}",
    input_variables=["text"]
)

merge_prompt = PromptTemplate(
    template="""Merge the following summary and questions into a single response.
    Format should be like this:
    Summary: {summary}
    Questions: {questions}
    """,
    input_variables=["summary", "questions"]
)

parallel_chain = RunnableParallel(
    {
        "summary": summary_prompt | gemini_model | StrOutputParser(),
        "questions": question_prompt | gemini_model | StrOutputParser()
    }
)

merge_chain = merge_prompt | nvidia_model | StrOutputParser()

final_parallel_chain = parallel_chain | merge_chain
final_parallel_response = final_parallel_chain.invoke({
    'text': "Hello, I am Abhijit Maharana. I am 22 years old and I am learning about Langchain and Langgraph to build RAG applications and Agentic AI models."
})

print(f"Final Parallel Chain Response: {final_parallel_response}")

In [ ]:
from pydantic import BaseModel, Field
from typing import Annotated, Literal
from langchain.schema.runnable import RunnableBranch, RunnableLambda
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser

# Conditional Chain
class ProductReview(BaseModel):
    product_feedback_from_user: Annotated[str, Field(description="Feedback provided by the user about the product")]
    sentiment_of_feedback: Annotated[Literal['Positive', 'Negative'], Field(description="Sentiment of the feedback provided by the user")]

string_parser = StrOutputParser()
pydantic_parser = PydanticOutputParser(pydantic_object=ProductReview)
    
sentiment_analysis_prompt = PromptTemplate(
    template = """Analyze the sentiment of the following feedback: 
    {feedback}
    
    Format instructions: {format_instructions}""",
    input_variables=["feedback"],
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
)

positive_feedback_prompt = PromptTemplate(
    template = "Give a polite reply for positive feedback of the customer response for the following feedback: {feedback}",
    input_variables=["feedback"]
)

negative_feedback_prompt = PromptTemplate(
    template = "Give a polite reply for negative feedback of the customer response for the following feedback: {feedback}",
    input_variables=["feedback"]
)

sentiment_chain = sentiment_analysis_prompt | gemini_model | pydantic_parser

feedback_chain = RunnableBranch(
    (lambda x: x.sentiment_of_feedback == "Positive", positive_feedback_prompt | gemini_model | string_parser),
    (lambda x: x.sentiment_of_feedback == "Negative", negative_feedback_prompt | gemini_model | string_parser),
    RunnableLambda(lambda x: "404 Error")
)

final_feedback_chain = sentiment_chain | feedback_chain
final_feedback_response = final_feedback_chain.invoke({
    "feedback": "The product very bad. Camera is not good at all!"
})

print(f"Final Feedback Response: {final_feedback_response}")

In [ ]:
from langchain.schema.runnable import RunnableSequence
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Runnables
# Runnable Sequence
runnable_sequence_prompt = PromptTemplate(
    template = """Tell me something about the topic: {topic}""",
    input_variables= ['topic']
)

# Define String output parser
parser = StrOutputParser()

# Define Runnable Sequence Chain
runnable_sequence_chain = RunnableSequence(runnable_sequence_prompt, gemini_model, parser)

runnable_sequence_response = runnable_sequence_chain.invoke({'topic': 'Future of AI'})
print(f"Runnable Sequence Ouput: {runnable_sequence_response}")

In [ ]:
from langchain.schema.runnable import RunnableParallel
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Runnable Parallel
question_prompt = PromptTemplate(
    template="""Make 3 questions and answers from the given text: {text}
    Format: 
    Question: 
    Answer:""",
    input_variables=["text"]
)

# Required Prompts
summary_prompt = PromptTemplate(
    template="Summarize the following text: {text}",
    input_variables=["text"]
)

runnable_parallel_chain = RunnableParallel(
    {
        "summary": summary_prompt | nvidia_model | StrOutputParser(),
        "questions": question_prompt | gemini_model | StrOutputParser()
    }
)

text = """Once upon a time, in a quiet village, there lived a kind girl named Ella.
One magical night, her fairy godmother appeared and transformed her rags into a beautiful gown so she could attend the royal ball.
At the ball, Ella danced with the prince, but at midnight she had to leave, losing her glass slipper on the stairs.
The prince searched the kingdom for the girl who fit the slipper, and
when he found Ella, they lived happily ever after."""

# Test chain
runnable_parallel_response = runnable_parallel_chain.invoke({'text': text})

print(f"Summary: {runnable_parallel_response['summary']}")
print(f"Questions: \n{runnable_parallel_response['questions']}")

In [ ]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Runnable Passthrough
topic_prompt = PromptTemplate(
    template = """Tell me something about the topic: {topic}""",
    input_variables= ['topic']
)

# Required Prompts
summary_prompt = PromptTemplate(
    template= """
    Summarize the following text:
    {text}
    """,
    input_variables=['text']
)

# Parser
parser = StrOutputParser()

# Topic Chain
topic_chain = topic_prompt | nvidia_model | parser

# Runnable Passthrough Chain
runnable_passthrough_chain = RunnableParallel({
    'topic_text': RunnablePassthrough(),
    'summary_text': summary_prompt | gemini_model | parser
})

# Final Chain
final_chain = topic_chain | runnable_passthrough_chain

runnable_passthrough_response = final_chain.invoke({'topic': 'blackhole'})
print(f"Topic Text:\n{runnable_passthrough_response['topic_text']}")
print(f"Summary Text: \n{runnable_passthrough_response['summary_text']}")



In [ ]:
from langchain.schema.runnable import RunnableLambda

# Runnable Lambda 
joke_prompt = PromptTemplate(
    template = "Tell me a small joke about the topic: {topic}",
    input_variables= ['topic']
)

# Define Function for lambda
def count_words(text):
    return len(text.split())

# Joke Chain
joke_chain = joke_prompt | gemini_model | parser

# Lambda Chain
lambda_chain = RunnableParallel({
    'word_count': RunnableLambda(count_words),
    'joke': RunnablePassthrough()
}
)

# Final Chain
final_runnable_lambda_chain = joke_chain | lambda_chain

runnable_lambda_response = final_runnable_lambda_chain.invoke({'topic': 'Programming'})
print(f"Word Count: {runnable_lambda_response['word_count']}")
print(f"Joke: \n{runnable_lambda_response['joke']}")

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableBranch, RunnableLambda, RunnablePassthrough, RunnableParallel

# Required Prompts
summary_prompt = PromptTemplate(
    template= "Summarize the following text: \n {text}",
    input_variables= ['text']
)

branched_chain = RunnableBranch(
    (lambda x: len(x.split()) > 20, summary_prompt | gemini_model | StrOutputParser()),
    RunnablePassthrough()
)


response = branched_chain.invoke("Hello My name is Abhijeet. I am 22 years old and I am doing AI ML Engineering. I build many rag based applications and Agentic AI workflows for Industry.")
print(response)